In [2]:
import os
import numpy as np
import pandas as pd
from shutil import copyfile

In [8]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=20,
                             shear_range=10,
                             validation_split=0.2)

train_generator = datagen.flow_from_directory('./origin1/',
                                              target_size=(28,28),
                                              subset='training')



Found 104 images belonging to 104 classes.
Found 0 images belonging to 104 classes.


In [14]:
from keras import backend as K
from keras import layers as L
from keras.models import Model,load_model
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

K.clear_session()

model_ckpt = ModelCheckpoint('BrailleNet.h5',save_best_only=True)
reduce_lr = ReduceLROnPlateau(patience=8,verbose=0)
early_stop = EarlyStopping(patience=15,verbose=1)

entry = L.Input(shape=(28,28,3))
x = L.SeparableConv2D(64,(3,3),activation='relu')(entry)
x = L.MaxPooling2D((2,2))(x)
x = L.SeparableConv2D(128,(3,3),activation='relu')(x)
x = L.MaxPooling2D((2,2))(x)
x = L.SeparableConv2D(256,(2,2),activation='relu')(x)
x = L.MaxPooling2D((2,2))(x)
x = L.SeparableConv2D(512,(2,2),activation='relu')(x)
x = L.GlobalMaxPooling2D()(x)
x = L.Dense(512)(x)
x = L.LeakyReLU()(x)
x = L.Dense(256,kernel_regularizer=l2(2e-4))(x)
x = L.LeakyReLU()(x)
x = L.Dense(104,activation='softmax')(x)

model = Model(entry,x)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

history = model.fit_generator(train_generator,
                              epochs=500,
                              verbose=1)

Epoch 1/500
4/4 [==============================] - 1s 42ms/step - loss: 4.7152 - accuracy: 0.0000e+00
Epoch 2/500
4/4 [==============================] - 0s 40ms/step - loss: 4.7071 - accuracy: 0.0195
Epoch 3/500
4/4 [==============================] - 0s 37ms/step - loss: 4.7010 - accuracy: 0.0038
Epoch 4/500
4/4 [==============================] - 0s 41ms/step - loss: 4.6949 - accuracy: 0.0303
Epoch 5/500
4/4 [==============================] - 0s 45ms/step - loss: 4.6887 - accuracy: 0.0357
Epoch 6/500
4/4 [==============================] - 0s 45ms/step - loss: 4.6819 - accuracy: 0.0199
Epoch 7/500
4/4 [==============================] - 0s 47ms/step - loss: 4.6655 - accuracy: 0.0275
Epoch 8/500
4/4 [==============================] - 0s 65ms/step - loss: 4.6172 - accuracy: 0.0587
Epoch 9/500
4/4 [==============================] - 0s 40ms/step - loss: 4.5127 - accuracy: 0.0459
Epoch 10/500
4/4 [==============================] - 0s 60ms/step - loss: 4.2778 - accuracy: 0.0521
Epoch 11/500
4/

In [21]:
model.save('BrailleNet_all_char.h5')

In [22]:
model = load_model('BrailleNet_all_char.h5')


In [23]:
from tensorflow.keras.preprocessing import image
img=image.load_img('./Braille Dataset/Braille Dataset/b1.JPG17whs.jpg')

In [24]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
model.predict(x)
a=np.argmax(model.predict(x), axis=1)


In [25]:
a

array([1], dtype=int64)